In [2]:
with open('cs273b_params.json') as params_open:
    params = json.load(params_open)
params_model = params['model']
params_train = params['train']

model = seqnn.SeqNN(params_model)
model.model.load_weights('model.h5')

NameError: name 'json' is not defined

In [1]:
import os
import h5py
import numpy as np
import keras
from keras import Input, Model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import Adam
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
import deeplift
from deeplift.conversion import kerasapi_conversion as kc

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
small_train_input = np.random.randint(1, 4, size=(10000, 1000, 4))
small_train_output = np.zeros((10000, 4))
print(len(small_train_input))
for i in range(len(small_train_input)):
    small_train_input[i:,0:20,:] = i%2
    small_train_output[i,:] = i%2

10000


In [3]:
# params = {'leftpos': 9500, 'rightpos':10500,
#           'numconvlayers': {'numFiltersConv1': 128, 'numFiltersConv2': 32,  'filterLenConv1':6, 'filterLenConv2': 9},
#          'dilRate1': 1,
#          'maxPool1': 30, 'dilRate2': 1, 'maxPool2': 10,
#          'numdenselayers': {'dense1': 64, 'dense2': 2, 'dropout1':0.00099, 'dropout2':0.01546},
#          'activationFxn': 'sigmoid',
#          'batch_size': 128}
#params = {'leftpos': 9500, 'rightpos':10500,
#          'numconvlayers': {'numFiltersConv1': 128, 'numFiltersConv2': 32,  'filterLenConv1':6, 'filterLenConv2': 9},
#         'dilRate1': 1,
#         'maxPool1': 30, 'dilRate2': 1, 'maxPool2': 10,
#         'numdenselayers': {'dense1': 64, 'dense2': 16, 'dropout1':0.00099, 'dropout2':0.01546},
#         'activationFxn': 'relu',
#         'batch_size': 128}
params = {'leftpos': 9500, 'rightpos':10500,
          'numconvlayers': {'numFiltersConv1': 128, 'numFiltersConv2': 144, 'numFiltersConv3': 162, 'numFiltersConv4': 202, 'numFiltersConv5': 253, 'filterLenConv1':17, 'filterLenConv2': 9, 'filterLenConv3-5': 5},
         'dilRate1': 1,
         'maxPool1': 5, 'dilRate2': 1, 'maxPool2': 5, 'maxPool3-5': 2, 'dilRate3-5': 1,
         'numdenselayers': {'dense1': 64, 'dense2': 16, 'dropout1':0.4, 'dropout2':0.3},
         'activationFxn': 'relu',
         'batch_size': 256}




In [4]:
# # leftpos = int(params['leftpos'])
# # rightpos = int(params['rightpos'])
# # X_trainpromoterSubseq = X_trainpromoter[:,leftpos:rightpos,:]
# # X_validpromoterSubseq = X_validpromoter[:,leftpos:rightpos,:]
# # input_promoter = Input(shape=X_trainpromoterSubseq.shape[1:], name='promoter')
# learning_rates = [0.0001]

# for lr in learning_rates:
#     input_promoter = Input(shape=small_train_input.shape[1:], name='promoter')

#     x = Conv1D(int(params['numconvlayers']['numFiltersConv1']), int(params['numconvlayers']['filterLenConv1']), dilation_rate=int(params['dilRate1']), padding='same', kernel_initializer='glorot_normal', input_shape=small_train_input.shape[1:],activation=params['activationFxn'])(input_promoter)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D(int(params['maxPool1']))(x)
#     x = Conv1D(int(params['numconvlayers']['numFiltersConv2']), int(params['numconvlayers']['filterLenConv2']), dilation_rate=int(params['dilRate2']), padding='same', kernel_initializer='glorot_normal',activation=params['activationFxn'])(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D(params['maxPool2'])(x)
#     x = Conv1D(int(params['numconvlayers']['numFiltersConv3']), int(params['numconvlayers']['filterLenConv3-5']), dilation_rate=int(params['dilRate3-5']), padding='same', kernel_initializer='glorot_normal',activation=params['activationFxn'])(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D(params['maxPool3-5'])(x)
#     x = Conv1D(int(params['numconvlayers']['numFiltersConv4']), int(params['numconvlayers']['filterLenConv3-5']), dilation_rate=int(params['dilRate3-5']), padding='same', kernel_initializer='glorot_normal',activation=params['activationFxn'])(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D(params['maxPool3-5'])(x)
#     x = Conv1D(int(params['numconvlayers']['numFiltersConv5']), int(params['numconvlayers']['filterLenConv3-5']), dilation_rate=int(params['dilRate3-5']), padding='same', kernel_initializer='glorot_normal',activation=params['activationFxn'])(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D(params['maxPool3-5'])(x)
#     x = Flatten()(x)
#     x = Dense(int(params['numdenselayers']['dense1']))(x)
#     x = Activation(params['activationFxn'])(x)
#     x = Dropout(params['numdenselayers']['dropout1'])(x)
#     #x = Dense(int(params['numdenselayers']['dense2']))(x)
#     #x = Activation(params['activationFxn'])(x)
#     #x = Dropout(params['numdenselayers']['dropout2'])(x)
#     x = Dense(4)(x)
#     main_output = Activation('softplus')(x) # force outputs to be at least 0
#     model = Model(inputs=[input_promoter], outputs=[main_output])
#     model.compile(Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),'poisson', metrics=['mean_squared_error'])
#     model.summary()
    
#     model.fit(x=small_train_input, y=small_train_output,epochs=10, shuffle='batch',
#               batch_size=params['batch_size'])
#     model.save('test_deeplift_model.h5')

In [5]:
from deepexplain.tensorflow import DeepExplain

In [6]:
class StochasticShift(tf.keras.layers.Layer):
  """Stochastically shift a one hot encoded DNA sequence."""
  def __init__(self, shift_max=0, pad='uniform', **kwargs):
    super(StochasticShift, self).__init__(**kwargs)
    self.augment_shifts = tf.range(-shift_max, shift_max+1)
    self.pad = pad
  def call(self, seq_1hot, training):
    def stoch_shift():
      shift_i = tf.random.uniform(shape=[], minval=0,
        maxval=len(self.augment_shifts), dtype=tf.int64)
      shift = tf.gather(self.augment_shifts, shift_i)

      sseq_1hot = tf.cond(tf.not_equal(shift, 0),
                          lambda: shift_sequence(seq_1hot, shift),
                          lambda: seq_1hot)
      return sseq_1hot

    return tf.cond(training,
                   stoch_shift,
                   lambda: seq_1hot)

class GELU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(GELU, self).__init__(**kwargs)
    def call(self, x):
        return tf.keras.activations.sigmoid(1.702 * x) * x
    
def shift_sequence(seq, shift, pad_value=0.25):
  """Shift a sequence left or right by shift_amount.

  Args:
  seq: [batch_size, seq_length, seq_depth] sequence
  shift: signed shift value (tf.int32 or int)
  pad_value: value to fill the padding (primitive or scalar tf.Tensor)
  """
  if seq.shape.ndims != 3:
      raise ValueError('input sequence should be rank 3')
  input_shape = seq.shape

  pad = pad_value * tf.ones_like(seq[:, 0:tf.abs(shift), :])

  def _shift_right(_seq):
    # shift is positive
    sliced_seq = _seq[:, :-shift:, :]
    return tf.concat([pad, sliced_seq], axis=1)

  def _shift_left(_seq):
    # shift is negative
    sliced_seq = _seq[:, -shift:, :]
    return tf.concat([sliced_seq, pad], axis=1)

  sseq = tf.cond(tf.greater(shift, 0),
                 lambda: _shift_right(seq),
                 lambda: _shift_left(seq))
  sseq.set_shape(input_shape)

  return sseq

class PearsonR(tf.keras.metrics.Metric):
  def __init__(self, num_targets, summarize=True, name='pearsonr', **kwargs):
    super(PearsonR, self).__init__(name=name, **kwargs)
    self._summarize = summarize
    self._shape = (num_targets,)
    self._count = self.add_weight(name='count', shape=self._shape, initializer='zeros')

    self._product = self.add_weight(name='product', shape=self._shape, initializer='zeros')
    self._true_sum = self.add_weight(name='true_sum', shape=self._shape, initializer='zeros')
    self._true_sumsq = self.add_weight(name='true_sumsq', shape=self._shape, initializer='zeros')
    self._pred_sum = self.add_weight(name='pred_sum', shape=self._shape, initializer='zeros')
    self._pred_sumsq = self.add_weight(name='pred_sumsq', shape=self._shape, initializer='zeros')

In [14]:
# #model = keras.models.load_model('model.h5')
# import numpy as np
# import tensorflow as tf
# from basenji import seqnn
# import json
# model = tf.keras.models.load_model('model.h5', custom_objects={'StochasticShift': StochasticShift, 'GELU': GELU, 
#                                                             'shift_sequence':shift_sequence, 'PearsonR': PearsonR})
with open('cs273b_params.json') as params_open:
    params = json.load(params_open)
params_model = params['model']
params_train = params['train']

model = seqnn.SeqNN(params_model)
model.model.load_weights('model.h5')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequence (InputLayer)        [(None, 1000, 4)]         0         
_________________________________________________________________
stochastic_shift (Stochastic (None, 1000, 4)           0         
_________________________________________________________________
gelu (GELU)                  (None, 1000, 4)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1000, 128)         8704      
_________________________________________________________________
batch_normalization (BatchNo (None, 1000, 128)         512       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 200, 128)          0         
_________________________________________________________________
gelu_1 (GELU)                (None, 200, 128)          0   

In [10]:
import shap
background = small_train_input[np.random.choice(small_train_input.shape[0], 100, replace=False)]
e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(small_train_input[200:201])

AssertionError: The model output must be a vector or a single value!

In [11]:
with DeepExplain(session=keras.backend.get_session()) as de:
    input_tensor = model.layers[0].input
    fModel = Model(inputs=input_tensor, outputs = model.layers[-2].output)
    target_tensor = fModel(input_tensor)
    
    xs = small_train_input[0:10]
    ys = small_train_input[0:10]
    attributions_gradin = de.explain('grad*input', target_tensor, input_tensor, xs, ys=ys)
    print(at)

RuntimeError: `get_session` is not available when TensorFlow is executing eagerly.

In [ ]:
import shap`
background = small_train_input[np.random.choice(small_train_input.shape[0], 100, replace=False)]
e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(small_train_input[200:205])


In [ ]:
shap_values

In [ ]:
deeplift_model =\
    kc.convert_model_from_saved_files(
        'test_deeplift_model.h5',
        nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.DeepLIFT_GenomicsDefault) 

In [ ]:
print(test_output.shape)
print(np.argwhere(test_output > 50))

In [ ]:
t = test_input[651910:651910+3]
print(t)
t.shape
print(model.predict(t))
print(test_output[651910:651910+3])

In [ ]:
test_input[36576]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(test_output[:20000,0], model.predict(test_input[:20000])[:,0])
plt.xlabel('Real Activation for Cell Type 0')
plt.ylabel('Predicted Activation for Cell Type 0')
plt.show()

In [ ]:
output_class = 2 # the index of the output class we want to maximize
output = model.layers[-1].output
target = tf.math.scalar_mul(2, output[:,output_class])
m = tf.math.reduce_mean(output, axis=1)
loss = m - target
grads = tf.gradients(loss, model.input)[0] # the output of `gradients` is a list, just take the first (and only) element
iterate = keras.backend.function([model.input], [loss, grads])

In [ ]:
def generate_seq():
    def adam_update(grad, i, w):
        #initialize the values of the parameters
        alpha = 1
        beta_1 = 0.9
        beta_2 = 0.999 
        epsilon = 1e-8

        nonlocal theta_0
        nonlocal m
        nonlocal v 

        m = beta_1 * m + (1 - beta_1) * grad
        v = beta_2 * v + (1 - beta_2) * np.power(grad, 2)
        m_hat = m / (1 - np.power(beta_1, i))
        v_hat = v / (1 - np.power(beta_2, i))
        w = w + alpha * m_hat / (np.sqrt(v_hat) + epsilon)
        return w

    theta_0 = 0
    m = 0 
    v = 0 
    
    raw_input_seq = np.random.random((1, 1000, 4)) # define an initial random image

    lr = 1.  # learning rate used for gradient updates
    max_iter = 1000  # number of gradient updates iterations
    for i in range(max_iter):
        loss_val, grads_val = iterate([raw_input_seq])
        #print(grads_val)
        raw_input_seq = adam_update(grads_val, i+1, raw_input_seq)
        
        #raw_input_seq += grads_val * lr  # update the image based on gradients
        if i % 100 == 0:
            pred = model.predict(raw_input_seq)
            print('Iteration ' + str(i) + ': model activation ' + str(pred))
    
    # Convert back to one-hot encoding
    raw_input_seq = np.squeeze(raw_input_seq)
    max_activations = np.argmax(raw_input_seq, axis=1).flatten()
    encoded_seq = np.zeros(raw_input_seq.shape)
    encoded_seq[np.arange(encoded_seq.shape[0]), max_activations] = 1
    
    return raw_input_seq, encoded_seq



In [ ]:
for i in range(2):
    input_seq, encoded_seq = generate_seq()
    print(input_seq[:10,])
    print(encoded_seq[:10,])